In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

#options.add_argument("--headless")  # Run Chrome in headless mode (no UI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

def get_youtube_titles(search_query):
    try:
        # Open YouTube
        driver.get("https://www.youtube.com")
        sleep(2)

        df = pd.DataFrame()

        for category in search_query:

            #print(category)

            # Find the search bar and enter query
            search_box = driver.find_element(By.NAME, "search_query")
            search_box.clear()
            search_box.send_keys(category)
            search_box.send_keys(Keys.RETURN)
            sleep(5)

            video_elements = driver.find_elements(By.XPATH, '//ytd-video-renderer')

            results = []

            cnt = 0
            while cnt < 50:
                sleep(1)
                video_elements = driver.find_elements(By.XPATH, '//ytd-video-renderer')
                for video in video_elements[cnt*2:cnt*2 + 2]:  
                    try:
                        title = video.find_element(By.XPATH, './/a[@id="video-title"]').text
                    except:
                        title = "N/A"

                    try:
                        thumbnail_element = (video
                                            .find_element(By.XPATH, './/a[@id="thumbnail"]//yt-image'))

                        driver.execute_script("arguments[0].scrollIntoView();", thumbnail_element)
                        sleep(1)
                        #thumbnail = driver.execute_script("return arguments[0].getAttribute('src');", thumbnail_element)
                        thumbnail = thumbnail_element.find_element(By.TAG_NAME, "img").get_attribute('src')
                    except:
                        thumbnail = "N/A"

                    try:
                        views = video.find_element(By.XPATH, './/span[contains(text(), "views")]').text
                    except:
                        views = "N/A"

                    try:
                        date = video.find_element(By.XPATH, './/span[contains(text(), "ago")]').text
                    except:
                        date = "N/A"

                    try:
                        href = video.find_element(By.XPATH, './/a[@id="thumbnail"]').get_attribute("href")
                        shorts =  "shorts" in href
                    except:
                        shorts = "N/A"

                    results.append({"title": title, "thumbnail": thumbnail, "views": views, 
                                    "date": date, "category": category, "shorts": shorts})
                    cnt += 1

            pd.DataFrame(results).to_csv(f"{category}.csv")

            df = pd.concat([df, pd.DataFrame(results)], axis = 0)

    finally:
        driver.quit()

    return df

search_term = ["Music", "Movies", "Books", "Games", "Sports", "Travel"]
df = get_youtube_titles(["Music", "Movies", "Books", "Games", "Sports", "Travel"])

In [12]:
for cat in search_term:
    print(df.loc[df["category"] == cat].shape)

(51, 5)
(50, 5)
(50, 5)
(51, 5)
(50, 5)
(50, 5)


In [15]:
df 

,title,thumbnail,views,date,category,shorts
0,"Lady Gaga, Bruno Mars - Die With A Smile (Offi...",https://i.ytimg.com/vi/kPa7bsKwL-c/hq720.jpg?s...,686M views,5 months ago,Music,False
1,Top hits 2024 playlist ~ Trending music 2024 ~...,https://i.ytimg.com/vi/U0ZoqmyGJo8/hq720.jpg?s...,45M views,6 months ago,Music,False
0,The Wish Swap - New Hallmark Movie 2025 🧡 Best...,https://i.ytimg.com/vi/EAFrndMF4OY/hq720.jpg?s...,3.9K views,8 hours ago,Movies,False
1,Client | Denzel Washington | Full Action Movie...,https://i.ytimg.com/vi/sCesFSomIrA/hq720.jpg?s...,271K views,6 days ago,Movies,False
0,"Oh, David! SMELLS! OOPS! | Children's Books Re...",https://i.ytimg.com/vi/qixOsybihFI/hq720.jpg?s...,1.3M views,6 years ago,Books,False
1,Come Book Shopping With me!! (first book haul ...,https://i9.ytimg.com/vi/MAdFjpkZ_EA/hq720_cust...,29K views,2 days ago,Books,False
0,Sean Avery on the INFAMOUS Avery Rule | Games ...,https://i.ytimg.com/vi/zc1AK4f5bDg/hq720.jpg?s...,29K views,2 days ago,Games,False
1,"State of Play | February 12, 2025 [English]",https://i.ytimg.com/vi/vQtPbTOhLEc/hq720.jpg?s...,2.2M views,Streamed 22 hours ago,Games,False
0,CRAZY 3-Sport Athlete!!! 😳 🤯 #shorts,https://i.ytimg.com/vi/qWAQ4jy9sbg/hq720_2.jpg...,18M views,2 years ago,Sports,True
1,Which sports should we do next? 😂,https://i.ytimg.com/vi/rNcMFBHikKc/hq720.jpg?s...,38M views,2 months ago,Sports,True
